<h1>TASK-1</h1>

In [1]:
import random
import uuid

categories = ["billing", "technical", "account", "general"]
weights = [0.4, 0.35, 0.15, 0.1]

tickets = []

for i in range(1, 251):
    ticket = {}
    
    ticket["ticket_id"] = f"TKT-{1000 + i}"
    ticket["customer_id"] = f"CUST-{random.randint(100, 999)}"
    ticket["category"] = random.choices(categories, weights=weights)[0]
    
    r = random.random()
    
    if r < 0.05:
        ticket["resolution_minutes"] = None
    elif r < 0.08:
        ticket["resolution_minutes"] = "unknown"
    else:
        ticket["resolution_minutes"] = random.randint(5, 300)
    
    ticket["escalated"] = random.random() < 0.2
    
    tickets.append(ticket)

print(tickets[:5])
print(len(tickets))


[{'ticket_id': 'TKT-1001', 'customer_id': 'CUST-462', 'category': 'general', 'resolution_minutes': 32, 'escalated': False}, {'ticket_id': 'TKT-1002', 'customer_id': 'CUST-760', 'category': 'billing', 'resolution_minutes': 'unknown', 'escalated': False}, {'ticket_id': 'TKT-1003', 'customer_id': 'CUST-722', 'category': 'billing', 'resolution_minutes': 29, 'escalated': False}, {'ticket_id': 'TKT-1004', 'customer_id': 'CUST-966', 'category': 'technical', 'resolution_minutes': 34, 'escalated': True}, {'ticket_id': 'TKT-1005', 'customer_id': 'CUST-492', 'category': 'account', 'resolution_minutes': 205, 'escalated': False}]
250


<h1>TASK-2</h1>

In [2]:
def check_required_keys(records):
    required = {"ticket_id", "customer_id", "category", "resolution_minutes", "escalated"}
    bad_indices = []
    for i, record in enumerate(records):
        if not required.issubset(record.keys()):
            bad_indices.append(i)
    return bad_indices


def find_invalid_resolution(records):
    bad_indices = []
    for i, record in enumerate(records):
        value = record.get("resolution_minutes")
        if not isinstance(value, int) or value < 0:
            bad_indices.append(i)
    return bad_indices


missing_key_records = check_required_keys(tickets)
invalid_resolution_records = find_invalid_resolution(tickets)

print("Records missing required keys:", missing_key_records[:10])
print("Invalid resolution_minutes records:", invalid_resolution_records[:10])
print("Total invalid resolution_minutes:", len(invalid_resolution_records))


Records missing required keys: []
Invalid resolution_minutes records: [1, 29, 30, 32, 35, 51, 55, 63, 76, 78]
Total invalid resolution_minutes: 19


<h1>TASK-3</h1>

In [3]:
def clean_tickets(records):
    cleaned = []
    
    for record in records:
        value = record.get("resolution_minutes")
        
        if not isinstance(value, int) or value < 0:
            continue
        
        new_record = {
            "ticket_id": record["ticket_id"],
            "customer_id": record["customer_id"],
            "category": record["category"].strip().lower(),
            "resolution_minutes": value,
            "escalated": record["escalated"]
        }
        
        cleaned.append(new_record)
    
    return cleaned


cleaned_tickets = clean_tickets(tickets)

print("Before cleaning:", len(tickets))
print("After cleaning:", len(cleaned_tickets))
print(cleaned_tickets[:5])



Before cleaning: 250
After cleaning: 231
[{'ticket_id': 'TKT-1001', 'customer_id': 'CUST-462', 'category': 'general', 'resolution_minutes': 32, 'escalated': False}, {'ticket_id': 'TKT-1003', 'customer_id': 'CUST-722', 'category': 'billing', 'resolution_minutes': 29, 'escalated': False}, {'ticket_id': 'TKT-1004', 'customer_id': 'CUST-966', 'category': 'technical', 'resolution_minutes': 34, 'escalated': True}, {'ticket_id': 'TKT-1005', 'customer_id': 'CUST-492', 'category': 'account', 'resolution_minutes': 205, 'escalated': False}, {'ticket_id': 'TKT-1006', 'customer_id': 'CUST-110', 'category': 'technical', 'resolution_minutes': 185, 'escalated': False}]


<h1>TASK-4</h1>

In [4]:
def average_resolution_per_category(records):
    totals = {}
    counts = {}
    
    for r in records:
        cat = r["category"]
        totals[cat] = totals.get(cat, 0) + r["resolution_minutes"]
        counts[cat] = counts.get(cat, 0) + 1
    
    return {cat: totals[cat] / counts[cat] for cat in totals}


def ticket_count_per_customer(records):
    counts = {}
    for r in records:
        cust = r["customer_id"]
        counts[cust] = counts.get(cust, 0) + 1
    return counts


def escalation_rates(records):
    total = len(records)
    total_escalated = 0
    
    cat_total = {}
    cat_escalated = {}
    
    for r in records:
        cat = r["category"]
        cat_total[cat] = cat_total.get(cat, 0) + 1
        
        if r["escalated"]:
            total_escalated += 1
            cat_escalated[cat] = cat_escalated.get(cat, 0) + 1
    
    overall_rate = total_escalated / total if total else 0
    
    by_category = {
        cat: cat_escalated.get(cat, 0) / cat_total[cat]
        for cat in cat_total
    }
    
    return {
        "overall_rate": overall_rate,
        "by_category": by_category
    }


avg_resolution = average_resolution_per_category(cleaned_tickets)
customer_counts = ticket_count_per_customer(cleaned_tickets)
escalation_summary = escalation_rates(cleaned_tickets)


category_validation = {}
for r in cleaned_tickets:
    category_validation[r["category"]] = category_validation.get(r["category"], 0) + 1

print("Category count sum:", sum(category_validation.values()))
print("Total cleaned:", len(cleaned_tickets))

print("Customer count sum:", sum(customer_counts.values()))
print("Total cleaned:", len(cleaned_tickets))

print("Average resolution:", avg_resolution)
print("Escalation summary:", escalation_summary)


Category count sum: 231
Total cleaned: 231
Customer count sum: 231
Total cleaned: 231
Average resolution: {'general': 119.10526315789474, 'billing': 150.9764705882353, 'technical': 161.87058823529412, 'account': 142.6904761904762}
Escalation summary: {'overall_rate': 0.20346320346320346, 'by_category': {'general': 0.15789473684210525, 'billing': 0.16470588235294117, 'technical': 0.2823529411764706, 'account': 0.14285714285714285}}


<h1>TASK-5</h1>

In [5]:
def build_report(records):
    avg_resolution = average_resolution_per_category(records)
    customer_counts = ticket_count_per_customer(records)
    escalation_summary = escalation_rates(records)
    
    report = {
        "total_tickets": len(records),
        "average_resolution_per_category": avg_resolution,
        "top_5_customers_by_ticket_count": dict(
            sorted(customer_counts.items(), key=lambda x: x[1], reverse=True)[:5]
        ),
        "escalation": escalation_summary
    }
    
    return report


final_report = build_report(cleaned_tickets)

print("=== SUPPORT TICKET REPORT ===")
print("Total tickets:", final_report["total_tickets"])
print("Average resolution per category:", final_report["average_resolution_per_category"])
print("Top 5 customers by ticket count:", final_report["top_5_customers_by_ticket_count"])
print("Escalation rates:", final_report["escalation"])

print("\nInsight:")
print("Technical and billing categories usually appear more often, and they tend to have higher escalation rates compared to general or account tickets.")


=== SUPPORT TICKET REPORT ===
Total tickets: 231
Average resolution per category: {'general': 119.10526315789474, 'billing': 150.9764705882353, 'technical': 161.87058823529412, 'account': 142.6904761904762}
Top 5 customers by ticket count: {'CUST-166': 3, 'CUST-504': 2, 'CUST-572': 2, 'CUST-421': 2, 'CUST-471': 2}
Escalation rates: {'overall_rate': 0.20346320346320346, 'by_category': {'general': 0.15789473684210525, 'billing': 0.16470588235294117, 'technical': 0.2823529411764706, 'account': 0.14285714285714285}}

Insight:
Technical and billing categories usually appear more often, and they tend to have higher escalation rates compared to general or account tickets.
